In [1]:
# Imports
import pyTigerGraph as tg
import json
import os
import pandas as pd
import xgboost as xgb
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.nn import GraphSAGE, GCN, GAT
from pyTigerGraph.gds.metrics import Accumulator
from pyTigerGraph.gds.metrics import Accuracy

# import TigerGraph instance config
os.chdir('../config/')
with open('tigergraph.json', 'r') as f:
    config = json.load(f)

# Connection parameters
hostName = config['host']
secret = config['secret']

conn = tg.TigerGraphConnection(host=hostName, gsqlSecret=secret, graphname="Ethereum")
conn.getToken(secret)

/Users/stephengelinas/.local/lib/python3.7/site-packages/pandas/compat/numpy/__init__.py:10: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  _nlv = LooseVersion(_np_version)
/Users/stephengelinas/opt/anaconda3/lib/python3.7/site-packages/setuptools/_distutils/version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/Users/stephengelinas/.local/lib/python3.7/site-packages/pandas/compat/numpy/function.py:120: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(__version__) >= LooseVersion("1.17.0"):
/Users/stephengelinas/.local/lib/python3.7/site-packages/pandas/compat/_optional.py:116: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if distutils.version.LooseVersion(version) < minimum_version:
/Users/stephengelinas/opt/anaconda3/lib/python3.7/site-packages

('f9r3r58ifp50em4olsofsca3hnng6ejh', 1679171225, '2023-03-18 20:27:05')

In [4]:
from pyTigerGraph.gds.metrics import Accuracy, BinaryPrecision, BinaryRecall
from collections import defaultdict

metrics = defaultdict(list)
m = Accuracy()
m = BinaryPrecision()
m = BinaryRecall()

In [6]:
hp = {"in_dim": 11, "hidden_dim": 32, "num_layers": 3, "lr": 0.005, "l2_penalty": 5e-5}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#data = data.to(device)

# initialize model
model = GAT(
    in_channels=hp["in_dim"],
    hidden_channels=hp["hidden_dim"],
    num_layers=hp["num_layers"],
    out_channels=hp["in_dim"],
    heads=8
).to(device)

# set optimizer
optimizer = torch.optim.Adam(
    model.parameters(), lr=hp["lr"], weight_decay=hp["l2_penalty"]
)

In [12]:
train_loader.data.is_train

tensor([ True,  True,  True,  ..., False, False, False])

In [ ]:
logs = {}
for epoch in range(100):
    # train model
    model.train()
    acc = Accuracy()
    # forward pass
    out = model(data.x.float(), data.edge_index)
    # compute loss
    loss = F.cross_entropy(out[data.is_train].float(), data.y[data.is_train].long())
    # backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # evaluate model on test set
    val_acc = Accuracy()
    with torch.no_grad():
        pred = out.argmax(dim=1)
        acc.update(pred[data.is_train], data.y[data.is_train])
        valid_loss = F.cross_entropy(out[data.is_test].float(), data.y[data.is_test].long())
        val_acc.update(pred[data.is_test], data.y[data.is_test])
    # logs
    logs["loss"] = loss.item()
    logs["test_loss"] = valid_loss.item()
    logs["acc"] = acc.value
    logs["test_acc"] = val_acc.value
    print("Epoch: {:02d}, Train Loss: {:.4f}, Test Loss: {:.4f}, Train Accuracy: {:.4f}, Test Accuracy: {:.4f}".format(
            epoch, logs["loss"], logs["test_loss"], logs["acc"], logs["test_acc"]))

In [ ]:
model.eval()
acc = Accuracy()
with torch.no_grad():
    pred = model(data.x.float(), data.edge_index).argmax(dim=1)
    acc.update(pred[data.is_test], data.y[data.is_test])
return acc.value